In [26]:
import pandas as pd # fun fact, pandas stands for PANel DAta

# 1. Import and inspect data

In [27]:
raw_csv_data = pd.read_csv('absenteeism_data.csv')

If we wanted to, we could force pd.options.display.max_rows (or max_volumns) to be equal to a number or None (shows everything!) to display more rows & columns of data respectively.

In [28]:
raw_csv_data.head(8)

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,11,26,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,36,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,3,23,15/07/2015,179,51,38,239.554,31,1,0,0,2
3,7,7,16/07/2015,279,5,39,239.554,24,1,2,0,4
4,11,23,23/07/2015,289,36,33,239.554,30,1,2,1,2
5,3,23,10/07/2015,179,51,38,239.554,31,1,0,0,2
6,10,22,17/07/2015,361,52,28,239.554,27,1,1,4,8
7,20,23,24/07/2015,260,50,36,239.554,23,1,4,0,4


In [29]:
raw_csv_data.describe()

,ID,Reason for Absence,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,17.951429,19.411429,222.347143,29.892857,36.417143,271.801774,26.737143,1.282857,1.021429,0.687143,6.761429
std,11.028144,8.356292,66.312960,14.804446,6.379083,40.021804,4.254701,0.668090,1.112215,1.166095,12.670082
min,1.000000,0.000000,118.000000,5.000000,27.000000,205.917000,19.000000,1.000000,0.000000,0.000000,0.000000
25%,9.000000,13.000000,179.000000,16.000000,31.000000,241.476000,24.000000,1.000000,0.000000,0.000000,2.000000
50%,18.000000,23.000000,225.000000,26.000000,37.000000,264.249000,25.000000,1.000000,1.000000,0.000000,3.000000
75%,28.000000,27.000000,260.000000,50.000000,40.000000,294.217000,31.000000,1.000000,2.000000,1.000000,8.000000
max,36.000000,28.000000,388.000000,52.000000,58.000000,378.884000,38.000000,4.000000,4.000000,8.000000,120.000000


# 2. Preprocess data

#### Always make a copy of your source data so as to leave it unaltered

In [30]:
df = raw_csv_data.copy()
df.shape

(700, 12)

## a. ID column (we can just drop this)

In [31]:
df = df.drop('ID', axis=1) # Axis=1 means columns
df.shape

(700, 11)

## b. Reason for absence column

In [32]:
df['Reason for Absence'].value_counts().sort_index()

0      38
1      16
2       1
3       1
4       2
5       3
6       6
7      13
8       5
9       4
10     22
11     24
12      8
13     52
14     18
15      2
16      3
17      1
18     21
19     36
21      6
22     32
23    147
24      3
25     29
26     31
27     66
28    110
Name: Reason for Absence, dtype: int64

The number 20 is missing! However this just means no-one has been absent with that reason.

We should encode these into dummy variables. To prevent perfect multicollinearity, we should always drop one dummy column (as having values of 0 for n-1 dummy columns implies that the dropped dummy column at position n has value 1).

In this case we want to drop reason 0 as a dummy column as it is actually doesn't have explanatory power - ie. it indicates a lack of a reason for absence.

In [33]:
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first=True)
reason_columns.shape

(700, 27)

As a result of this, we need to then drop the Reason for Absence column in our df.

In [34]:
df = df.drop('Reason for Absence', axis=1)

But, adding 27 new columns to our table seems excessive, we should group similar types. See here for details on the groups https://www.udemy.com/course/the-data-science-course-complete-data-science-bootcamp/learn/lecture/12486122.

We want to group reasons 1-14 as diseases, 15-17 as pregnancy, 18-21 as severe illnesses, and 22-28 as "lighter" reasons (eg. dentist).

In [35]:
reason_columns.loc[:, 1:14] # Remember, loc is INCLUSIVE
# reason_columns.iloc[:, 1:14] # This returns 13 columns as iloc is NOT INCLUSIVE of final column

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,0,0,0,0,0,0,0,0,1,0,0,0,0
696,0,0,0,0,0,1,0,0,0,0,0,0,0,0
697,0,0,0,0,0,0,0,0,0,1,0,0,0,0
698,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We create new columns accordingly by taking the max value for each row in each group.

In [36]:
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

reason_types_simple = pd.concat([
    reason_type_1,
    reason_type_2,
    reason_type_3,
    reason_type_4],
    axis=1)

Concatenate new reason_type columns to df

In [37]:
df = pd.concat([df, reason_types_simple], axis=1)
df.head(3)

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,0,1,2,3
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1


We should rename the dummy columns.

In [38]:
df = df.rename(columns={
    0: "Reason_1", 
    1: "Reason_2", 
    2: "Reason_3", 
    3: "Reason_4"
    })

df.head(3)

,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Reason_1,Reason_2,Reason_3,Reason_4
0,07/07/2015,289,36,33,239.554,30,1,2,1,4,0,0,0,1
1,14/07/2015,118,13,50,239.554,31,1,1,0,0,0,0,0,0
2,15/07/2015,179,51,38,239.554,31,1,0,0,2,0,0,0,1


Then reorder columns to put target at the end.

In [39]:
df = df[['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']]

df.head(3)

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,07/07/2015,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,14/07/2015,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,15/07/2015,179,51,38,239.554,31,1,0,0,2


##### We can create a checkpoint to save progress rather than having to run everything from the start 

In [40]:
df_reason_mod = df.copy()

## c. Date column

In [41]:
# All data in a column is of same type, so 0 index is representative
type(df_reason_mod['Date'][0])

str

- %d day
- %m month
- %Y year
- %H hour
- %M minute
- %S second

In [42]:
# Format specifies INPUT timestamp format, not output!
df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format='%d/%m/%Y')
df_reason_mod.loc[5,'Date']

Timestamp('2015-07-10 00:00:00')

We want to extract months from the dates.

In [43]:
list_months = [df_reason_mod['Date'][i].month for i in range(df_reason_mod.shape[0])]
print(len(list_months), min(list_months), max(list_months))
df_reason_mod['Month'] = list_months

700 1 12


Then day of week.

In [44]:
df_reason_mod['Day of Week'] = df_reason_mod['Date'].apply(lambda date: date.weekday())
df_reason_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Month,Day of Week
0,0,0,0,1,2015-07-07,289,36,33,239.554,30,1,2,1,4,7,1
1,0,0,0,0,2015-07-14,118,13,50,239.554,31,1,1,0,0,7,1
2,0,0,0,1,2015-07-15,179,51,38,239.554,31,1,0,0,2,7,2
3,1,0,0,0,2015-07-16,279,5,39,239.554,24,1,2,0,4,7,3
4,0,0,0,1,2015-07-23,289,36,33,239.554,30,1,2,1,2,7,3


Finally, remove the date and reorder the columns

In [45]:
df_reason_mod = df_reason_mod.drop('Date', axis=1)
df_reason_mod = df_reason_mod[['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4',
       'Month', 'Day of Week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']]

And set another checkpoint

In [46]:
df_reason_date_mod = df_reason_mod.copy()
df_reason_date_mod.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month,Day of Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,1,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,1,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,1,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,1,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,1,2,1,2


## d. Education

_Note, we will not be changing the expense - BMI columns as there's nothing to be done with these._

- 1: high school
- 2: graduate
- 3: postgraduate
- 4: master/doctor

In [47]:
df_reason_date_mod['Education'].value_counts().sort_index()

1    583
2     40
3     73
4      4
Name: Education, dtype: int64

Given that education levels above 1 are fairly infrequent, we should overwrite the other values to a single category.

In [48]:
df_reason_date_mod['Education'] = [0 if x==1 else 1 for x in df_reason_date_mod['Education']]

In [49]:
df_reason_date_mod['Education'].value_counts()

0    583
1    117
Name: Education, dtype: int64

## Final checkpoint

In [50]:
df_preprocessed = df_reason_date_mod.copy()
df_preprocessed.to_csv('absenteeism_data_preprocessed.csv', index=False)